# Introdução a Otimização Linear

**Técnica criada por George B. Dantzig em 1947 e que se propõe a otimizar (maximizar ou minimizar) o valor de uma função linear, respeitando um conjunto de restrições (equações ou inequações) lineares.**


> O principal solver de otimização linear do OR-Tools é o Glop, o sistema de programação linear do Google. É rápido, eficiente em termos de memória e numericamente estável. A próxima seção mostra como usar o Glop para resolver um problema linear simples.

## Um exemplo simples
Aqui está um exemplo simples de um problema de otimização linear.

**Maximize 3x + 4y sujeito às seguintes restrições:**
\begin{equation}
x + 2y	\le	14\\
3x – y	\le	0\\
x – y	\le	2\\
\end{equation}

Tanto a função objetivo, $3x + 4y$, quanto as restrições são dadas por expressões lineares, o que torna um problema linear.

As restrições definem a região viável, que é o triângulo mostrado abaixo, incluindo seu interior.
<img src="img/feasible_region.png">

## As seções a seguir explicam como resolver o problema passo a passo.
0. Incluir as bibliotecas

In [1]:
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

1. Escolha o solver.
> O código a seguir declara o solucionador. O MPsolver é um invólucro para vários solucionadores diferentes, incluindo o Glop. Escolha GLOP_LINEAR_PROGRAMMING para informar ao solucionador para usar Glop.

In [2]:
MPSolver solver("ExemploLinear", MPSolver::GLOP_LINEAR_PROGRAMMING);

2. Crie as variáveis.
> crie variáveis x e y cujos valores estão no intervalo de 0 a infinito. Ou seja, x e y são variáveis não negativas.

In [3]:
auto const infinity = solver.infinity();

auto const x = solver.MakeNumVar(0.0, infinity, "x");
auto const y = solver.MakeNumVar(0.0, infinity, "y");

std::cout << "Número de variáveis = " << solver.NumVariables();

Número de variáveis = 2

3. Defina as restrições.
> defina as restrições das variáveis. Atribua a cada restrição um nome exclusivo (como restrição 0) e defina os coeficientes para a restrição.

In [4]:
// x + 2y <= 14.
auto const c0 = solver.MakeRowConstraint(-infinity, 14.0);
c0->SetCoefficient(x, 1);
c0->SetCoefficient(y, 2);

// 3x - y >= 0.
auto const c1 = solver.MakeRowConstraint(0.0, infinity);
c1->SetCoefficient(x, 3);
c1->SetCoefficient(y, -1);

// x - y <= 2.
auto const c2 = solver.MakeRowConstraint(-infinity, 2.0);
c2->SetCoefficient(x, 1);
c2->SetCoefficient(y, -1);
std::cout << "Número de restrições = " << solver.NumConstraints();

Número de restrições = 3

4. Defina a função objetivo.
> O código a seguir define a função objetivo, 3x + 4y, e especifica que este é um problema de maximização.

In [5]:
// FO: 3x + 4y.
auto const objective = solver.MutableObjective();
objective->SetCoefficient(x, 3);
objective->SetCoefficient(y, 4);
objective->SetMaximization();

5. Execute o solver e exiba os resultados.
> O código a seguir chama o solver e exibe a solução.

In [6]:
auto const result_status = solver.Solve();

if (result_status != MPSolver::OPTIMAL) {
  std::cout << "O problema não tem uma solução ótima!";
}

std::cout << "Solução:\n";
std::cout << "Valor ótimo da FO = " << objective->Value() << std::endl;
std::cout << x->name() << " = " << x->solution_value() << std::endl;
std::cout << y->name() << " = " << y->solution_value() << std::endl;

Solução:
Valor ótimo da FO = 34
x = 6
y = 4


## O programa completo

Agora escreva o programa completo na caixa de código abaixo e verifique os resultados.

In [7]:
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

{
    
}

O programa deve retornar a solução ótima para o problema, como mostrado abaixo.

Solução:
- x = 6
- y = 4

Valor ótimo: FO = 34

**A linha verde tracejada exibida no gráfico abaixo é encontrada definindo a função objetivo igual ao seu valor ótimo de 34.**

> Qualquer linha cuja equação tenha a forma **3x + 4y = c** é paralela à linha tracejada e 34 é o maior valor de **c** para o qual a linha cruza a região viável.

<img src="img/feasible_region_solution.png">

**Se você pensar na geometria no gráfico acima, em qualquer problema de otimização linear, pelo menos um vértice da região viável deve ser uma solução ótima.**

> Você poderia então criar um algoritmo para encontrar uma solução ótima percorrendo os vértices da região viável até que não haja mais melhorias na função objetivo. Essa é exatamente a idéia por trás do algoritmo **simplex**, o método mais usado para resolver problemas de otimização linear.



# Solucionando Problemas de Programação Linear: O Método Simplex

# A Teoria do Método Simplex